In [61]:
import sys
import os
from preproc import get_list
import matplotlib.pyplot as plt



##**load data**

In [ ]:
bigram = sc.parallelize(get_list())
topic_list = sorted(bigram.map(lambda x:x[0]).distinct().collect())
number_of_topic=len(topic_list)
print number_of_topic

##**compute match divide by volume**

In [ ]:
def match_divide_by_volume(bigram):
    new_bigram = bigram.map(lambda x:((x[0],x[2]),(x[3],x[4])))
    reduced_bigram = new_bigram.reduceByKey(lambda a,b:map(sum,zip(a,b)))
    mdbv = reduced_bigram.map(lambda x:(x[0],float(x[1][0])/x[1][1]))
    return mdbv

mdbv = match_divide_by_volume(bigram)

top_15=mdbv.takeOrdered(15,lambda x:-x[1])
print top_15

##**get volume over all years**

In [ ]:
def volume_over_all_years(bigram):
    new_bigram = bigram.map(lambda x:((x[0],x[2]),x[4]))
    voay = new_bigram.reduceByKey(lambda a,b:a+b)
    return voay

voay = volume_over_all_years(bigram)

print top_15


##**get match over all years**

In [ ]:
def match_over_all_years(bigram):
    new_bigram = bigram.map(lambda x:((x[0],x[2]),x[3]))
    moay = new_bigram.reduceByKey(lambda a,b:a+b)
    return moay
moay = match_over_all_years(bigram)
top_15=moay.takeOrdered(15,lambda x:-x[1])
print top_15


##**plot**

In [ ]:
for i in range (0,number_of_topic):
    topic_v=voay.filter(lambda x:x[0][0]==topic_list[i])
    topic_m=moay.filter(lambda x:x[0][0]==topic_list[i])
    topic_mv=mdbv.filter(lambda x:x[0][0]==topic_list[i])
    count=topic_v.count()
    ordered_v=topic_v.sortBy(lambda x:x[0][1]).collect()
    ordered_m=topic_m.sortBy(lambda x:x[0][1]).collect()
    ordered_mv=topic_mv.sortBy(lambda x:x[0][1]).collect()

    year = [ordered_v[i][0][1] for i in range (0,count)]
    match = [ordered_m[i][1] for i in range (0,count)]
    volume = [ordered_v[i][1] for i in range (0,count)]
    match_db_volume = [ordered_mv[i][1] for i in range (0,count)]
    
    
    fig=plt.figure()
    plt.plot(year,match)
    plt.plot(year,volume)
    plt.legend(['Match','Volume'])
    plt.xlabel('year')
    plt.ylabel('number')
    plt.show()
    
    plt.plot(year,match_db_volume)
    plt.xlabel('year')
    plt.ylabel('match divided by volume')
    plt.show()